# Bayesian Network

## Inhalt
1. Initialisierung und Datenimport
2. Design des Bayesian Networks (Graph)
3. Berechnen der CPTs
4. Implementierung des Bayesian Networks
5. Test und Evaluation
6. Fazit

## 1. Initialisierung und Datenimport

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from pomegranate import * # <--- muss ggf. mit pip install installiert werden

In [ ]:
data = pd.read_csv('Versicherung_A1.csv', sep=';')
data.head()

## 2. Design des Bayesian Networks (Graph)

Gegeben sind 9 Variablen: Geschlecht, Familienstand, Alter, Kinder, Bildungsstand, Beruf, Jahresgehalt, Immobilienbesitz und Versicherungstarif.

In diesem Kapitel möchten wir schrittweise ein 'sinnvolles' probabilistisches Netz entwickeln. Wir starten mit folgenden Annahmen:
1. Unsere Zielvariable ist der **Versicherungstarif** (A, B, C oder abgelehnt).
2. Die einzigen Variablen, welche unabhängig von allen anderen Variablen sind und somit keine Elternknoten im Graph haben werden, sind **Alter** und **Geschlecht**.

Somit ergibt sich der folgende Graph.

![title](img/bayes-start.png)

Nachfolgend untersuchen wir mithilfe von Datenvisualiserungstechniken, wie sich das **Alter** und **Geschlecht** auf die verbleibenden Variablen auswirkt.

In [ ]:
unknown_vars = ['Bildungsstand', 'Familienstand', 'Beruf', 'Kinder', 'Jahresgehalt', 'Immobilienbesitz']

for var in unknown_vars:
    if var == 'Jahresgehalt':
        pass
    else:
        sns.catplot(x=var, y='Alter', data=data)
        sns.catplot(x=var, hue='Geschlecht', data=data, kind='count')

In [ ]:
# Geschlecht vs Gehalt
data.groupby('Geschlecht').describe()['Jahresgehalt']

Aus den obigen Plots lesen wir folgende Dinge ab:
1. Die Verteilungen der einzelnen Kategorien sind stark geprägt durch das **Alter** der Personen. Wir gehen davon aus, dass das Alter aufgrund der zugrundeliegenden Biologie und gesetzlichen Normen der direkte Elternknoten von jedem einzelnen anderen Knoten ist.
2. Insbesondere können wir beim **Alter** mehrere 'Levels' erkennen: Personen unter 20 Jahre und Personen über 65 Jahre können jeweils gesondert betrachtet werden.
3. Das **Geschlecht** macht meist nur einen geringen Unterschied aus. Allerdings können wir auch hier eine leicht ungleichmäßige Verteilung feststellen, insbesondere bei **Bildungsstand**, **Immobilienbesitz** und **Beruf**.

Weiterhin treffen wir folgende Annahmen:
4. Ein weiterer Elternknoten von **Beruf** ist der **Bildungsstand** der Person.
5. Die Elternknoten von **Gehalt** sind neben dem **Alter** der **Bildungsstand** und der **Beruf** der Person. Wir stellen ebenfalls einen leichten Gender Pay Gap fest, können jedoch nicht ausschließen, ob dieser direkt durch das Geschlecht oder bereits durch die ungleichmäßigen Verteilungen beim Bildungsstand und Beruf entstanden ist.
6. Ein weiterer Elternknoten von **Immobilienbesitz** ist das **Jahresgehalt**.

Nachfolgend wollen wir bestimmen, welche Variablen sich auf den Tarif auswirkt. Zunächst betrachten wir die numerischen Variablen und können aus den folgenden vier Plots folgende Dinge ablesen:
7. Niemand unter 20 Jahre erhält einen Tarif (alle unter 20 werden abgelehnt).
8. Niemand mit einem Gehalt unter 10000 erhält einen Tarif und nur Personen mit einem hohen Gehalt erhalten Tarif A.
9. Niemand mit mehr als 2 Kindern erhält Tarif **A**. Wir denken jedoch nicht, dass die Kinderanzahl ausschlaggebend für den Tarif ist, sondern der Besitz einer Immobilie. Der letzte Plot zeigt, dass große Ähnlichkeiten zwischen Kinder-vs.-Tarif und Kinder-vs.-Immobilienbesitz bestehen.

Wir schließen daraus, dass das Alter und Gehalt direkte Elternknoten für den Tarif sind \[1, 7, 8\], Kinder jedoch nicht \[9\]. Wir nehmen außerdem Kinder als zusätzlichen Elternknoten für Immobilien auf, da diese Graphen beinahe gleich sind \[9\].

In [ ]:
cont_vars = ['Alter', 'Jahresgehalt', 'Kinder']

for cont in cont_vars:
    sns.catplot(x='Versicherungstarif', y=cont, data=data)
    
sns.catplot(x='Immobilienbesitz', y='Kinder', data=data)

Es ergibt sich folgender Graph:
![title](img/bayes-mid.png)

Für die restlichen (kategorischen) Variablen berechnen wir die bedingte Wahrscheinlichkeit, einen der Tarife zu bekommen, gegeben dass jeweils eine der restlichen Variablen eintritt. Dadurch soll verhindert werden, dass wir einen Elternknoten für Tarife aufnehmen, der eigentlich besser nur indirekt wirkt (Großvater/Urgroßvater/...).

Für die kategorischen Variablen benutzen wir one-hot encoding, sodass für jede Kategorie eine einzelne True/False Spalte erzeugt wird.

In [ ]:
# one-hot encoding
data_dummies = pd.get_dummies(data)
data_dummies.head()

$P(A|B) = \frac{P(A und B)}{P(B)}$	

Ereignis A ist unsere Zielvariable, also Tarif A, Tarif B, Tarif C, oder ablehnen.  
Ereignis B sind unsere unabhängigen Variablen; für's erste wir nehmen hierfür alle categorical Variablen.  
  
Wir iterieren über alle Zielvariablen und unabhängigen Variablen, und berechnen für jede Kombination die bedingte Wahrscheinlichkeit.

In [ ]:
data_dummies.columns

In [ ]:
# categorical input variables
cat_columns = ['Geschlecht_maennlich', 'Geschlecht_weiblich', 'Familienstand_geschieden',
       'Familienstand_ledig', 'Familienstand_verheiratet',
       'Familienstand_verwitwet', 'Bildungsstand_Abitur',
       'Bildungsstand_Bachelor', 'Bildungsstand_Hauptschule',
       'Bildungsstand_Keine', 'Bildungsstand_Lehre', 'Bildungsstand_Master',
       'Bildungsstand_Promotion', 'Bildungsstand_Realschule',
       'Beruf_angestellt', 'Beruf_arbeitend', 'Beruf_im Ruhestand',
       'Beruf_kein', 'Beruf_selbständig', 'Beruf_verbeamtet',
       'Immobilienbesitz_Eigentumswohnung', 'Immobilienbesitz_Einfamilienhaus',
       'Immobilienbesitz_Mehrfamilienhaus', 'Immobilienbesitz_Villa',
       'Immobilienbesitz_keiner']

In [ ]:
total = len(data)
total

In [ ]:
# Calculating P(A)

target_variables = ['Versicherungstarif_Tarif A',
       'Versicherungstarif_Tarif B', 'Versicherungstarif_Tarif C',
       'Versicherungstarif_ablehnen']

p_A = {}

for target in target_variables:
    p_A[target] = data_dummies[target].sum()/total
    
p_A

In [ ]:
# Calculating P(B)

p_B = {}

for cat in cat_columns:
    p_B[cat] = data_dummies[cat].sum()/total
    
p_B

In [ ]:
# Calculating P(A and B)

p_A_and_B = {}

for target in target_variables:
    for cat in cat_columns:
        b = data_dummies[cat] == 1
        a = data_dummies[target] == 1
        p_A_and_B[f'{target}_and_{cat}'] = len(data_dummies[a&b])/total
        
p_A_and_B

In [ ]:
# Calculating P(A|B) = P(A and B)/P(B)

p_A_given_B = {}

for target in target_variables:
    for cat in cat_columns:
        p_A_given_B[f'{target}_given_{cat}'] = p_A_and_B[f'{target}_and_{cat}']/p_B[cat]

        
p_A_given_B

Wir testen mit folgender Formel, ob manche der Zielereignisse unabhängig (oder nur schwach abhängig) von Inputvariablen sind und filtern diese Inputvariablen heraus. Diese Inputvariablen können nicht mehr direkte Elternknoten für den Tarif werden.

$P(A|B) \approx P(A)$; 

Oder etwas präziser: wir definieren die Ereignisse A und B als unabhängig ("nicht genug abhängig"), falls $0.8 < \frac{P(A|B)}{P(A)} < 1.25$


In [ ]:
# Checking for stochastic independence
for target in target_variables:
    for cat in cat_columns:
        ratio = p_A_given_B[f'{target}_given_{cat}'] / p_A[target]
        if 4/5 < ratio < 5/4:
            print(f'{target[19:]} - {cat}')

Aus obiger Liste ziehen wir folgende Schlüße:
1. Die Tarife sind unabhängig vom **Geschlecht**. Somit ist **Geschlecht** kein direkter Elternknoten vom Tarif.
2. Manche Tarife sind teilweise unabhängig von **Familienstand** und **Bildungsstand**. Wir entscheiden uns dazu, den Familienstand als Elternknoten vom Tarif zu behalten; den Bildungsstand aber nicht als Elternknoten aufzunehmen, da der Bildungsstand noch indirekt über Gehalt und Beruf mit dem Tarif verwandt ist.
3. Alle Tarife *sind abhängig* von den kategorischen Variablen **Immobilienbesitz** und **Beruf**, sodass wir diese zwei Knoten als Elternknoten einbauen.

Es ergibt sich folgender finaler Graph.
![title](img/bayes-end.png)


# 3. Berechnen der CPTs

Mit folgender Funktion *P()* und Klasse *Vertex* können wir sehr einfach die bedingten Wahrscheinlichkeiten für alle Nodes ausrechnen (CPT).

In [ ]:
import itertools

def P(query):
    total = len(data)
    x = data.query(query)
    return len(x)/total

# P('Geschlecht=="maennlich"')
# >>> 0.502

class Vertex:
    
    def __init__(self, name, s):
        self.name = name
        self.parents = []
        self.states = s
        self.cpt = None
        
    # Adds a new parent to the node
    def register_parent(self, vertex):
        self.parents.append(vertex)
        
    # Updates the CPT
    def update_cpt(self):
        
        states = [];

        # Check if node has parents
        if len(self.parents) > 0:
            
            for p in self.parents:
                states.append(p.states)
                
            states.append(self.states)
            
            permutations_tuples = list(itertools.product(*states))
            self.cpt = []
            
            for p in permutations_tuples:
                self.cpt.append(list(p))
                            
            for row in self.cpt:
                given = " & ".join(row[:-1])
                
                if P(given) == 0:
                    row.append(0)
                else:
                    prob = P(" & ".join(row))/P(given)
                    row.append(prob)
                            
        # If node has no parents, just calculate the 'normal' probability P(s)
        else:
            states = [[s] for s in self.states]
            for s in states:
                s.append(P(s[0]))
            self.cpt = states
            
    def print_cpt(self):
        columns = [p.name for p in self.parents]
        columns.append(self.name)
        columns.append('Prob')
        return pd.DataFrame(self.cpt, columns = columns)

### Alter
Wie zuvor angedeutet möchten wir das Alter in 3 Klassen aufteilen:
1. Personen unter 20
2. Personen zwischen 20 und 65
3. Personen >= 65

Hierzu rechnen wir die CPT aus (ohne condition, da keine parent node da ist) und überprüfen das Ergebnis kurz von Hand.

In [ ]:
age = Vertex('Alter', ['Alter < 20', 'Alter >=20 & Alter < 65', 'Alter>=65'])
age.update_cpt()
age.print_cpt()

In [ ]:
# Herausfiltern aller Personen unter 20 Jahre
u20 = data[data['Alter'] < 20]
# Anzahl der herausgefilterten Datensätze durch Gesamtzahl teilen
print(len(u20)/len(data))

# Gleiches für die beiden anderen Altersgruppen
ue20 = data[(data['Alter'] >= 20) & (data['Alter'] < 65)]
print(len(ue20)/len(data))

ue65 = data[data['Alter'] >= 65]
print(len(ue65)/len(data))

# ==> Ergebnis stimmt mit obiger Tabelle überein

### Geschlecht
Ähnlich zu Alter (ohne Elternknoten).

In [ ]:
sex = Vertex('Geschlecht', ['Geschlecht=="maennlich"', 'Geschlecht=="weiblich"'])
sex.update_cpt()
sex.print_cpt()

### Familienstand

Wir berechnen die bedingte Wahrscheinlichkeit für Familienstand und Alter und überprüfen das Ergebnis stichprobenartig.  
$P(Familienstand | Alter)$

In [ ]:
familienstand = Vertex('Familienstand', ['Familienstand=="ledig"', 'Familienstand=="geschieden"',
                      'Familienstand=="verwitwet"', 'Familienstand=="verheiratet"'])
familienstand.register_parent(age)
familienstand.update_cpt()
familienstand.print_cpt()

In [ ]:
# P(A|B) = P(A und B)/P(B)
# A: ledig
# B: <20 Jahre
# ==> Erste Zeile im DataFrame

print('P(A und B)')
print(P('Familienstand=="ledig" & Alter < 20'))
l_u20 = data[(data['Alter'] < 20) & (data['Familienstand'] == 'ledig')]
print(len(l_u20)/len(data))

print('\nP(B)')
print(P('Alter < 20')) ### Haben wir bereits ganz oben beim Alter überprüft --> 0.122


print('P(A|B) = P(A und B)/P(B)')
print(P('Familienstand=="ledig" & Alter < 20')/P('Alter < 20')) # --> Stimmt mit Ergebnis in Tabelle überein

## Kinder
Wir teilen die Personen in zwei Gruppen auf. :
1. Personen mit <3 Kindern
2. Personen mit >=3 Kindern


Diese Grenze begründen wir dadurch, dass Personen mit >= 3 Kindern keine Villa besitzen und auch keinen Tarif A bekommen.

In [ ]:
kinder = Vertex('Kinder', ['Kinder < 3', 'Kinder >= 3'])
kinder.register_parent(age)
kinder.update_cpt()
kinder.print_cpt()

## Bildungsstand
Wir berechnen die bedingte Wahrscheinlichkeit des Bildungsstands (gegeben Alter und Geschlecht) und überprüfen das Ergebnis stichprobenartig.

In [ ]:
states = [f'Bildungsstand=="{x}"' for x in data['Bildungsstand'].unique()]

education = Vertex('Bildungsstand', states)
education.register_parent(age)
education.register_parent(sex)
education.update_cpt()
education.print_cpt()

In [ ]:
# Test, ob P(Bachelor | 20-65 & weiblich) stimmt --> Zeile 25 im Dataframe.
p_a_und_b = P('Bildungsstand=="Bachelor" & Alter >=20 & Alter < 65 & Geschlecht=="weiblich"')
p_b = P('Alter >=20 & Alter < 65 & Geschlecht=="weiblich"')
print(p_a_und_b / p_b) # ---> Stimmt

## Beruf

In [ ]:
states = [f'Beruf=="{x}"' for x in data['Beruf'].unique()]

beruf = Vertex('Beruf', states)
beruf.register_parent(age)
beruf.register_parent(sex)
beruf.register_parent(education)
beruf.update_cpt()
beruf.print_cpt()

## Jahresgehalt
Wir erstellen erneut mehrere diskrete Gruppen basierend auf den Erkenntnissen im ersten Teil:
1. Personen mit < 10.000
2. Personen mit >= 10.000 & < 60.000
3. Personen mit >= 60.000 & < 80.000
4. Personen mit >= 80.000 & < 100.000
5. Personen mit > 100.000

In [ ]:
states =  ['Jahresgehalt < 10000', 'Jahresgehalt >= 10000 & Jahresgehalt < 60000',
           'Jahresgehalt >= 60000 & Jahresgehalt < 80000',
           'Jahresgehalt >= 80000 & Jahresgehalt < 100000',
           'Jahresgehalt >= 100000']

gehalt = Vertex('Jahresgehalt', states)
gehalt.register_parent(age)
gehalt.register_parent(education)
gehalt.register_parent(beruf)
gehalt.update_cpt()
gehalt.print_cpt()

## Immobilienbesitz

In [ ]:
states = [f'Immobilienbesitz=="{x}"' for x in data['Immobilienbesitz'].unique()]


immobilie = Vertex("Immobilienbesitz", states)
immobilie.register_parent(age)
immobilie.register_parent(sex)
immobilie.register_parent(kinder)
immobilie.register_parent(gehalt)
immobilie.update_cpt()
immobilie.print_cpt()

## Tarif

In [ ]:
states = [f'Versicherungstarif=="{x}"' for x in data['Versicherungstarif'].unique()]

tarif = Vertex("Versicherungstarif", states)
tarif.register_parent(age)
tarif.register_parent(familienstand)
tarif.register_parent(beruf)
tarif.register_parent(gehalt)
tarif.register_parent(immobilie)
tarif.update_cpt()
tarif.print_cpt()


# 4. Implementierung des Bayesian Networks

Für das Bayesian Network benutzen wir die Bibliothek *pomegranate*. Zuerst müssen für alle Variablen die möglichen Ereignisse/Ausgänge und dazugehörige (bedingte) Wahrscheinlichkeiten angegeben. Danach werden entsprechende Nodes erzeugt, einem Modell hinzugefügt und miteinander verbunden. 

In [ ]:
# Defining probability distributions
sex_cpt = DiscreteDistribution({'Geschlecht=="maennlich"': 0.502, 'Geschlecht=="weiblich"': 0.498})
age_cpt = DiscreteDistribution({"Alter < 20": 0.122, "Alter >=20 & Alter < 65": 0.764, "Alter>=65": 0.114})
familienstand_cpt = ConditionalProbabilityTable(familienstand.cpt, [age_cpt])
kinder_cpt = ConditionalProbabilityTable(kinder.cpt, [age_cpt])
bildungsstand_cpt = ConditionalProbabilityTable(education.cpt, [age_cpt, sex_cpt])
beruf_cpt = ConditionalProbabilityTable(beruf.cpt, [age_cpt, sex_cpt, bildungsstand_cpt])
gehalt_cpt = ConditionalProbabilityTable(gehalt.cpt, [age_cpt, bildungsstand_cpt, beruf_cpt])
immobilie_cpt = ConditionalProbabilityTable(immobilie.cpt, [age_cpt, sex_cpt, kinder_cpt, gehalt_cpt])
tarif_cpt = ConditionalProbabilityTable(tarif.cpt, [age_cpt, familienstand_cpt, beruf_cpt, gehalt_cpt, immobilie_cpt])

# Defining nodes
sex_n = Node(sex_cpt, name="Geschlecht")
age_n = Node(age_cpt, name="Alter")
familienstand_n = Node(familienstand_cpt, name="Familienstand")
kinder_n = Node(kinder_cpt, name="Kinder")
bildungsstand_n = Node(bildungsstand_cpt, name="Bildungsstand")
beruf_n = Node(beruf_cpt, name="Beruf")
gehalt_n = Node(gehalt_cpt, name="Gehalt")
immobilie_n = Node(immobilie_cpt, name="Immobilienbesitz")
tarif_n = Node(tarif_cpt, name="Versicherungstarif")

# Init model
model = BayesianNetwork("Versicherungstarif_Netz")

# Add states
model.add_states(sex_n, familienstand_n, age_n,  kinder_n, bildungsstand_n, beruf_n, gehalt_n, immobilie_n, tarif_n)

# Add edges
# familienstand und kinder
model.add_edge(age_n, familienstand_n) 
model.add_edge(age_n, kinder_n) 

# Bildungsstand
model.add_edge(age_n, bildungsstand_n)
model.add_edge(sex_n, bildungsstand_n)

# Beruf
model.add_edge(age_n, beruf_n)
model.add_edge(sex_n, beruf_n)
model.add_edge(bildungsstand_n, beruf_n)

# Gehalt
model.add_edge(age_n, gehalt_n)
model.add_edge(bildungsstand_n, gehalt_n)
model.add_edge(beruf_n, gehalt_n)

# Immobilie
model.add_edge(age_n, immobilie_n)
model.add_edge(sex_n, immobilie_n)
model.add_edge(kinder_n, immobilie_n)
model.add_edge(gehalt_n, immobilie_n)

# Immobilie
model.add_edge(age_n, tarif_n)
model.add_edge(familienstand_n, tarif_n)
model.add_edge(beruf_n, tarif_n)
model.add_edge(gehalt_n, tarif_n)
model.add_edge(immobilie_n, tarif_n)


model.bake()

In [ ]:
# Beispiele
model.predict([['Geschlecht=="maennlich"', None, 'Alter < 20', None, None, None, None, None, None]])

In [ ]:
model.predict_proba([['Geschlecht=="maennlich"', 'Familienstand=="verheiratet"', 'Alter >=20 & Alter < 65', 'Kinder < 3', 
                      'Bildungsstand=="Master"', 'Beruf=="selbständig"', 'Jahresgehalt >= 100000', 
                      'Immobilienbesitz=="Einfamilienhaus"', None]])

# 5. Test und Evaluation

Mit folgendem Beispielcode können Sie die fehlenden Werte von Personen vorhersagen.

In [ ]:
# Hilfsklasse für Person
class Person:
    
    def __init__(self, df_row):
        self.geschlecht = df_row[0]
        self.familienstand = df_row[1]
        self.alter = df_row[2]
        self.kinder = df_row[3]
        self.bildungsstand = df_row[4]
        self.beruf = df_row[5]
        self.jahresgehalt = df_row[6]
        self.immobilienbesitz = df_row[7]
        self.versicherungstarif = df_row[8]
        
    def get_bayes_query(self):
        query = []
        query.append(None if self.geschlecht==None else f'Geschlecht=="{self.geschlecht}"')
        query.append(None if self.familienstand==None else f'Familienstand=="{self.familienstand}"')
        query.append(None if self.alter==None else 'Alter < 20' if self.alter < 20 
                     else 'Alter >=20 & Alter < 65' if 20 <= self.alter < 65 
                     else 'Alter>=65')
        query.append(None if self.kinder==None else 'Kinder < 3' if self.kinder < 3 else 'Kinder >= 3')
        query.append(None if self.bildungsstand==None else f'Bildungsstand=="{self.bildungsstand}"')
        query.append(None if self.beruf==None else f'Beruf=="{self.beruf}"')
        query.append(None if self.jahresgehalt==None else 'Jahresgehalt < 10000' if self.jahresgehalt < 10000
                     else 'Jahresgehalt >= 10000 & Jahresgehalt < 60000' if 10000 <= self.jahresgehalt < 60000
                     else 'Jahresgehalt >= 60000 & Jahresgehalt < 80000' if 60000 <= self.jahresgehalt < 80000
                     else 'Jahresgehalt >= 80000 & Jahresgehalt < 100000' if 80000 <= self.jahresgehalt < 100000
                     else 'Jahresgehalt >= 100000')
        query.append(None if self.immobilienbesitz==None else f'Immobilienbesitz=="{self.immobilienbesitz}"')
        query.append(None if self.versicherungstarif==None else f'Versicherungstarif=="{self.versicherungstarif}"')
        
        return query

In [ ]:
# Testdatensatz
data_test = pd.read_csv('test_daten.csv', sep=";")
data_test = data_test.where(pd.notnull(data_test), None)
data_test.head()

In [ ]:
# Ausgabe der vorhergesagten Werte
for index, row in data_test.iterrows():
    print(index)
    p = Person(data_test.iloc[index,:])
    print(model.predict([p.get_bayes_query()]))

In [ ]:
# Vergleich mit echten Werten
data.head()

## 6. Fazit

Mithilfe von Pomegranate und verschiedener Hilfsklassen konnten wir ein Bayesian Network entwickeln, das fehlende Variablen vorhersagt. Basierend auf Erkenntnissen, die durch Datenvisualisierung und Unabhängigkeitstests gewonnen wurden, haben wir zunächst die Beziehungen der Variablen untersucht und einen passenden Graph entwickelt. Daraufhin konnten wir die Conditional Probability Tables automatisiert berechnen und das Netzwerk implementieren und testen.

Das Design des Graphs ist maßgeblich für die weitere Berechnung und Implementierung der Conditional Probability Tables und des Bayesian Netzwerk. Es gibt einige Entscheidungen, über die man sicherlich streiten kann:
- sollte **Kinder** ein Elternknoten von **Immobilienbesitz** sein (-> muss man sich als Erwachsener entscheiden, ob man ein Kind oder ein Haus will?)
- ist das **Geschlecht** nicht doch direkt mitverantwortlich für den **Tarif** (Erfahrungen aus der echten Welt würden dies wohl bestätigen).
- ab wann sind Ereignisse unabhängig? Ist 0.8 < P(A|B)/P(A) < 1.25 ein zu hoher Schwellwert für stochastische Abhängigkeit?
- wurden die numerischen Variablen (Alter, Kinder, Gehalt) in zu große Gruppen eingeteilt? Sollte man hier feinere Abstufungen implementieren oder gar eine kontinuierliche Eingabe/Vorhersage ermöglichen?
- sollte die CPT an Stellen, an denen der Nenner P(A|B)/**P(B)** null war (und somit die bedingte Wahrscheinlichkeit aufgrund von mangelnden Beispielen nicht berechnet werden konnte), mit Nullen aufgefüllt werden oder sollten diese Zeilen von der Tabelle gelöscht werden?
- ...

Für viele dieser Designfragen versuchten wir, eine möglichst klare Antwort aus den Daten ablesen zu können. Wir sind zuversichtlich, ein passendes Netzwerk implementiert zu haben.